
# Generating Bash Code with Granite Code and Ollama


### Prerequisite: Install Ollama and Granite Code models

1. [Download and Start Ollama](https://ollama.com/download)
1. Install Granite Code 20b: `ollama pull granite-code:20b`
1. Install Granite Code 8b: `ollama pull granite-code:8b`

In [1]:
!pip install ollama


## One-shot Prompt with Granite Code 20b

In One-shot prompting, you provide the model with a question and no examples. The model will generate an answer given its training. Larger models tend to do better at this task.

Use the [ollama-python package](https://github.com/ollama/ollama-python) to access the model.

In [1]:
import ollama

Let's write two helper functions that we'll use for all our queries. First, we'll find it useful to determine the name of our operating system and use that string in queries. This is because shell commands sometimes have different options on Linux vs. MacOS, etc. We'll write our queries so they take this difference into account. 

In [14]:
import platform

def os_name():
    os_name = platform.system()
    # It turns out, using "MacOS" is better than "Darwin", which is what gets returned on MacOS. 
    # For all other cases, the returned value should be fine as is, so we map the result to the desired
    # name, but only for MacOS...
    name_map = {'Darwin': 'MacOS'}
    # ... then pass the os_name value as the second arg, which is used as the default return value.
    return name_map.get(os_name, os_name)  

In [15]:
print(f"My OS is {os_name()}")

My OS is MacOS


Now let's write a helper function for running queries, wrapping the Ollama API. Note the default model name we'll used, Granite Code 20B (parameters). Also, note how we add additional content to the user's input prompt, such "make sure you write code that works for _my_ system!"

In [26]:
def query(prompt: str, model: str = 'granite-code:20b') -> str:
    response = ollama.generate(
        model=model, 
        prompt=f"{prompt}. Make sure you generate code that is {os_name()}-compatible!")
    print(response["response"])
    return response["response"]

In [27]:
query("""
    Show me a bash script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time 
    for each file in the output.""")

Here's an example of a bash script that will achieve this task:

```bash
#!/bin/bash
find . -type f -mtime -7 | head -n 50 | xargs ls -lt
```

This script uses the `find` command to search for files (`-type f`) within the current working directory and its subdirectories. The `-mtime -7` option limits the search to files modified within the last week (i.e., 7 days). The `head -n 50` command selects the first 50 matching files, and `xargs ls -lt` displays the last modification time for each of these files.

Note that this script assumes a Mac-based system as it uses the `ls` command with the `-lt` option to display the last modification time in a human-readable format. If you are using a different operating system, you may need to adjust the final command accordingly.


Paste the command in the next cell. _**Keep the `!` shown**_, e.g., `!ls -al`, so Jupyter knows to run the command as a shell script instead of Python. (You can omit lines like `#!/bin/bash`.) 

Does the script work? If not try running the query again. Try modifying the query string, too.

In [18]:
!find . -type f -mtime -7 | head -n 50 | xargs ls -lta

-rw-r--r--@ 1 deanwampler  staff  15366 Aug 27 17:03 ./Text_to_Shell.ipynb
-rw-r--r--@ 1 deanwampler  staff  26285 Aug 27 16:51 ./.ipynb_checkpoints/Text_to_Shell-checkpoint.ipynb


## Few-shot Prompting with Granite Code 8b

In few-shot prompting, you provide the model with a question and some examples. The model will generate an answer given its training. The additional examples help the model zero in on a pattern, which may be required for smaller models to perform well at this task.

In [32]:
examples = """
Question:
Recursively finds files like '*.js', and filters out files with 'excludeddir' in path.
Answer:
find . -name '*.js' | grep -v excludeddir

Question:
Dump \"a0b\" as hexadecimal bytes
Answer:
printf \"a0b\" | od -tx1

Question:
create a tar ball of all pdf files in current folder
Answer:
find . -name *.pdf | xargs tar czvf pdf.tar

Question:
Sort all files/directories in the current directory, but no subdirectories, according to modification time and print only the recent 7 of them
Answer:
find . -maxdepth 1 -exec stat -f "%Sm {}" \; | sort -n -r | tail -n 7

Question:
find all the empty directories in the current folder
Answer:
find . -type d -empty

"""

Let's define another helper function for calling `ollama.chat()`

In [33]:
def chat1(prompt: str, examples: str = examples, model: str ='granite-code:8b') -> str:
    user_prompt = f"""
        {examples}
        Question:
        {prompt}. Make sure you generate code that is {os_name()}-compatible!
        Answer:"""
    response = ollama.chat(model=model, messages=[
      {
        'role': 'user',
        'content': user_prompt
      },
    ])
    answer = response['message']['content']
    print(answer)
    return answer

In [34]:
chat1("""
    Show me a bash script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time 
    for each file in the output.""")

find . -type f -mtime -7 | head -n 50


'find . -type f -mtime -7 | head -n 50'

## Adding a System Prompt

A system prompt can be used to provide additional instructions and clarity or context for the task. Here we let the model know what we expect from it.

Let's define a final helper function that uses a system prompt.

In [37]:
default_system_prompt = f"""
    You are a helpful software engineer. You write clear, concise, well-commented code."""

def chat(prompt: str, 
         system_prompt:str = default_system_prompt,
         examples: str = examples, 
         model: str ='granite-code:8b') -> str:
    user_prompt = f"""
        {examples}
        Question:
        {prompt} Make sure you only generate code that is {os_name()}-compatible!
        Answer:""" 
    response = ollama.chat(model=model, messages=[
      {
        'role':'system',
        'content': system_prompt  
      },
      {
        'role': 'user',
        'content': user_prompt
      },
    ])
    answer = response['message']['content']
    print(answer)
    return answer

In [39]:
chat("""
    Show me a bash script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time 
    for each file in the output.""")

Here's a bash script that prints the first 50 files found under the current working directory that have been modified within the last week, along with their last modification times:

    find . -type f -mmin -1440 | head -n 50 | xargs ls -lth
    Explanation:

        The 'find' command is used to search for files in the current directory and its subdirectories that meet certain criteria. In this case, we are looking for files (indicated by '-type f') that have been modified within the last 7 days (-mmin -1440).
        
        The output of the 'find' command is then piped to the 'head' command, which limits the output to the first 50 lines. This ensures that we only see the most recently modified files.
        
        Finally, the 'xargs' command is used to pass each file name to the 'ls -lth' command, which prints a detailed list of the files along with their last modification times in human-readable format.


"Here's a bash script that prints the first 50 files found under the current working directory that have been modified within the last week, along with their last modification times:\n\n    find . -type f -mmin -1440 | head -n 50 | xargs ls -lth\n    Explanation:\n\n        The 'find' command is used to search for files in the current directory and its subdirectories that meet certain criteria. In this case, we are looking for files (indicated by '-type f') that have been modified within the last 7 days (-mmin -1440).\n        \n        The output of the 'find' command is then piped to the 'head' command, which limits the output to the first 50 lines. This ensures that we only see the most recently modified files.\n        \n        Finally, the 'xargs' command is used to pass each file name to the 'ls -lth' command, which prints a detailed list of the files along with their last modification times in human-readable format."